# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

## TODO Explore data

In [2]:
# load in the data
azdias = pd.read_csv('../data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../data/Udacity_CUSTOMERS_052018.csv', sep=';')

print(azdias.shape)
azdias.head()

/home/gabor/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(891221, 366)


,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [6]:
# Read up data dictionary
df_dictionary = pd.read_excel('../data/DIAS Attributes - Values 2017.xlsx', skiprows=1, usecols=[1,2,3,4])
df_dictionary['Attribute'].fillna(method='ffill', inplace=True)
df_dictionary.loc[df_dictionary['Value']=='…', 'Type'] = 'Numerical'
df_dictionary.loc[~(df_dictionary['Value']=='…'), 'Type'] = 'Non-Numerical'

# list of possible values
possible_values = df_dictionary.groupby(by='Attribute')['Value'].apply(list)
data_type = df_dictionary[['Attribute', 'Type']].drop_duplicates()
print(len(possible_values), len(data_type))
data_type = data_type.merge(possible_values, left_on='Attribute', right_index=True).reset_index(drop=True)
print(len(data_type))
# missing values
df_missing = df_dictionary[df_dictionary['Meaning'].astype('str').str.contains('unknown')]
print(len(data_type))
data_type = data_type.merge(df_missing[['Attribute', 'Value']], left_on='Attribute', right_on='Attribute', how='left').reset_index(drop=True)
print(len(data_type))
data_type.rename(columns={'Value_x':'Values','Value_y':'Missing values'}, inplace=True)
data_type.head(2)

314 314
314
314
314


,Attribute,Type,Values,Missing values
0,AGER_TYP,Non-Numerical,"[-1, 0, 1, 2, 3]",-1
1,ALTERSKATEGORIE_GROB,Non-Numerical,"[-1, 0, 1, 2, 3, 4, 9]","-1, 0"


In [45]:
cols_customers = customers.columns
cols_azdias = azdias.columns
cols_datadict = sorted(data_type['Attribute'])
print(len(cols_customers))
print(len(cols_azdias))
print(len(cols_datadict))

df_cols = pd.Series(sorted(cols_customers), name='customers').to_frame()
df_cols = df_cols.merge(pd.Series(sorted(cols_azdias), name='azdias').to_frame(), left_on='customers', right_on='azdias', how='outer')

# Deal with mispelled column names
#rename the _RZ
cols_datadict = [col[0:-3] if col[-3:]=='_RZ' else col for col in cols_datadict ]
df_cols.loc[df_cols['customers']=='D19_BUCH_CD'] = 'D19_BUCH'

# KBA13_CCM_1400_2500 -> KBA13_CCM_1401_2500
df_dict_cols = pd.Series(sorted(cols_datadict), name='dictionary').to_frame()
df_dict_cols.loc[df_dict_cols['dictionary']=='KBA13_CCM_1400_2500']='KBA13_CCM_1401_2500'

# CAMEO_DEUINTL_2015 -> CAMEO_INTL_2015
df_dict_cols.loc[df_dict_cols['dictionary']=='CAMEO_DEUINTL_2015']='CAMEO_INTL_2015'

# SOHO_FLAG -> SOHO_KZ
df_dict_cols.loc[df_dict_cols['dictionary']=='SOHO_FLAG']='SOHO_KZ'

df_cols = df_cols.merge(df_dict_cols, left_on='customers', right_on='dictionary', how='outer')

df_cols['All Cols']=df_cols.fillna(method='pad', axis=1)['dictionary']
len(df_cols)

369
366
314


374

In [56]:
cols_to_drop = df_cols[df_cols.isnull().any(axis=1)]['All Cols'].values
print(len(cols_to_drop))
cols_to_drop[:12]

65


array(['AKT_DAT_KL', 'ALTERSKATEGORIE_FEIN', 'ALTER_KIND1', 'ALTER_KIND2',
       'ALTER_KIND3', 'ALTER_KIND4', 'ANZ_KINDER',
       'ANZ_STATISTISCHE_HAUSHALTE', 'ARBEIT', 'CJT_KATALOGNUTZER',
       'CJT_TYP_1', 'CJT_TYP_2'], dtype=object)

## I'll need columns present in both dataset at the same time.

In [65]:
# Drop all non matching columns
cols_to_drop_azdias = [col for col in cols_to_drop if col in azdias.columns ]
print(len(cols_to_drop_azdias))
azdias_coldropped = azdias.drop(columns=cols_to_drop_azdias)

57


## TODO Clean data

In [89]:
data_type.set_index('Attribute')['Missing values'].str.split(',').to_dict()

{'AGER_TYP': nan,
 'ALTERSKATEGORIE_GROB': ['-1', ' 0'],
 'ALTER_HH': nan,
 'ANREDE_KZ': ['-1', ' 0'],
 'ANZ_HAUSHALTE_AKTIV': nan,
 'ANZ_HH_TITEL': nan,
 'ANZ_PERSONEN': nan,
 'ANZ_TITEL': nan,
 'BALLRAUM': nan,
 'BIP_FLAG': nan,
 'CAMEO_DEUG_2015': nan,
 'CAMEO_DEU_2015': nan,
 'CAMEO_DEUINTL_2015': nan,
 'CJT_GESAMTTYP': nan,
 'D19_BANKEN_ANZ_12': nan,
 'D19_BANKEN_ANZ_24': nan,
 'D19_BANKEN_DATUM': nan,
 'D19_BANKEN_DIREKT_RZ': nan,
 'D19_BANKEN_GROSS_RZ': nan,
 'D19_BANKEN_LOKAL_RZ': nan,
 'D19_BANKEN_OFFLINE_DATUM': nan,
 'D19_BANKEN_ONLINE_DATUM': nan,
 'D19_BANKEN_ONLINE_QUOTE_12': nan,
 'D19_BANKEN_REST_RZ': nan,
 'D19_BEKLEIDUNG_GEH_RZ': nan,
 'D19_BEKLEIDUNG_REST_RZ': nan,
 'D19_BILDUNG_RZ': nan,
 'D19_BIO_OEKO_RZ': nan,
 'D19_BUCH_RZ': nan,
 'D19_DIGIT_SERV_RZ': nan,
 'D19_DROGERIEARTIKEL_RZ': nan,
 'D19_ENERGIE_RZ': nan,
 'D19_FREIZEIT_RZ': nan,
 'D19_GARTEN_RZ': nan,
 'D19_GESAMT_ANZ_12': nan,
 'D19_GESAMT_ANZ_24': nan,
 'D19_GESAMT_DATUM': nan,
 'D19_GESAMT_OFFLINE_DATUM

In [91]:
azdias.dtypes.unique()

array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

In [69]:
# Loop through columns, check if values are in the right range and convert missing values to NaN
for col in azdias_coldropped.columns:
    print('{}: {}'.format(col, azdias_coldropped[col].unique()))


AGER_TYP: [-1  2  3  0  1]
ALTER_HH: [nan  0. 17. 13. 20. 10. 14. 16. 21. 11. 19. 15.  9. 18.  8.  7. 12.  4.
  3.  6.  5.  2.  1.]
ANZ_HAUSHALTE_AKTIV: [ nan  11.  10.   1.   3.   5.   4.   6.   2.   9.   0.  14.  13.   8.
   7.  44.  25.  36.  16. 272.  33.  15.  12.  22.  21.  20.  67.  23.
  18.  27.  24.  31.  39.  19.  30.  17.  29.  61.  54.  88.  49.  60.
  38.  34.  48.  35.  62.  26.  45.  42.  28.  77.  47.  50. 236. 137.
  43.  52.  41.  76.  63.  32.  40.  37.  59.  53.  57. 118.  66.  95.
  78.  70.  46. 185.  73. 154.  80. 102.  92. 126.  94.  71.  68.  81.
  69. 121. 258.  58.  86. 124.  85.  90. 128.  51.  72.  64.  56.  83.
 150. 259.  91. 160. 134. 169. 246.  82. 114. 183. 116.  74. 143.  99.
 216.  98. 125. 112. 123.  55. 120.  75. 141. 103.  87.  89. 223.  93.
 179. 135. 149.  97. 101. 110.  79. 111. 109.  65.  84. 367. 162. 195.
 107. 119. 331. 122. 161. 157. 175. 177. 136. 104. 316. 127. 100. 180.
 155. 133. 178. 176. 108. 353. 131. 130.  96. 256. 117. 146. 211. 

KBA13_ALTERHALTER_60: [nan  3.  5.  2.  4.  1.]
KBA13_ALTERHALTER_61: [nan  4.  3.  2.  5.  1.]
KBA13_ANZAHL_PKW: [ nan 963. 712. ...   2.  30.   7.]
KBA13_AUDI: [nan  4.  3.  5.  2.  1.]
KBA13_AUTOQUOTE: [nan  2.  3.  4.  5.  1.  0.]
KBA13_BJ_1999: [nan  3.  2.  4.  1.  5.]
KBA13_BJ_2000: [nan  3.  2.  1.  4.  5.]
KBA13_BJ_2004: [nan  3.  4.  2.  1.  5.]
KBA13_BJ_2006: [nan  3.  5.  2.  4.  1.]
KBA13_BJ_2008: [nan  3.  4.  0.  2.  5.  1.]
KBA13_BJ_2009: [nan  2.  1.  3.  5.  0.  4.]
KBA13_BMW: [nan  3.  4.  2.  5.  1.]
KBA13_CCM_0_1400: [nan  2.  1.  3.  5.  0.  4.]
KBA13_CCM_1000: [nan  0.  1.  4.  5.  3.  2.]
KBA13_CCM_1200: [nan  0.  2.  1.  4.  5.  3.]
KBA13_CCM_1400: [nan  4.  2.  3.  1.  5.]
KBA13_CCM_1401_2500: [nan  3.  2.  1.  4.  5.]
KBA13_CCM_1500: [nan  1.  4.  3.  5.  2.]
KBA13_CCM_1600: [nan  2.  3.  1.  4.  5.]
KBA13_CCM_1800: [nan  2.  4.  3.  5.  1.  0.]
KBA13_CCM_2000: [nan  5.  3.  4.  2.  1.]
KBA13_CCM_2500: [nan  3.  4.  0.  5.  2.  1.]
KBA13_CCM_2501: [nan  3.  4

## Compare to population

Similar to Kolmogorov-Smirnov Test
1. loop through each column
  1. calculate relative proportions of each category
    - for both population and costumers
  2. Calculate difference for each
  3. Sum the absolute value
  
2. Order columns by the biggest difference 

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')